<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/%EC%96%91%EC%9E%90%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_70_Short_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98(%EC%96%91%EC%9E%90%EC%86%8C%EC%9D%B8%EC%88%98%EB%B6%84%ED%95%B4_%EB%B0%98%EB%B3%B5%EC%A3%BC%EA%B8%B0%EB%B0%9C%EA%B2%AC)_20240416.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
!python3 -m pip install qiskit[visualization]
!python3 -m pip install qiskit-aer

In [76]:
import qiskit
import qiskit_aer

print(qiskit.__version__)
print(qiskit_aer.__version__)

1.0.2
0.14.0.1


In [77]:
import math
import random
import numpy as np
from fractions import Fraction
from qiskit import *
#from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
#from qiskit.circuit import Gate, QuantumCircuit, QuantumRegister, ParameterVector

from qiskit_aer import Aer
import matplotlib.pyplot as plt
#from qiskit.circuit import Gate
#from qiskit.circuit.library import CU1Gate
#from qiskit.quantum_info import Statevector
#from qiskit.visualization import plot_histogram

#from qiskit.circuit.library import QFT
#from qiskit.quantum_info.operators import Operator

#from qiskit.compiler import assemble

In [78]:
def factorize4(N):
    trial = 0
    while (True):
        trial += 1
        print('trial =', trial)
        a = random.randint(2, N - 1)
        if a not in [2, 7, 8, 11, 13]:
            continue
        r, qc = findPeriodByQuantumCircuit(N, a)
        print('\ta =', a, 'r =', r)
        if (r % 2 != 0):
            continue
        gcd1 = math.gcd(N, a ** (r // 2) + 1)
        gcd2 = math.gcd(N, a ** (r // 2) - 1)
        print('\tgcd1 =', gcd1, 'gcd2 =', gcd2)
        if (gcd1 == 1 or gcd2 == 1):
            continue
        return gcd1, gcd2, qc


In [79]:
def findPeriodByQuantumCircuit(N, a):
    phase, qc = qpe_amod15(a)
    frac = Fraction(phase).limit_denominator(15)
    return frac.denominator, qc

In [80]:
def qpe_amod15(a):
    n_count = 3
    qc = QuantumCircuit(4 + n_count, n_count)
    for q in range(n_count):
        qc.h(q)
    qc.x(3 + n_count)
    for q in range(n_count):
        qc.append(c_amod15(a, 2 ** q), [q] + [i + n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count))
    qc.measure(range(n_count), range(n_count))

    backend = Aer.get_backend('qasm_simulator')
    result = backend.run(qc, shots=1024, memory=True).result()
    readings = result.get_memory()
    phase = int(readings[0], 2) / (2 ** n_count)
    return phase, qc

In [81]:
def c_amod15(a, power):
    if a not in [2, 7, 8, 11, 13]:
        raise ValueError("'a' must be in [2, 7, 8, 11, 13]")
    U = QuantumCircuit(4)
    for iteration in range(power):
        if a in [2, 13]:
            U.swap(0, 1)
            U.swap(1, 2)
            U.swap(2, 3)
        if a in [7, 8]:
            U.swap(2, 3)
            U.swap(1, 2)
            U.swap(0, 1)
        if a == 11:
            U.swap(1, 3)
            U.swap(0, 2)
        if a in [7, 11, 13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = " %i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

In [82]:
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi / float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = " QFT† (I-QFT)"
    return qc

In [83]:
def main():
    N = 3 * 5
    p, q, qc = factorize4(N)
    print(N, '=', p, '*', q)
    qc.draw()

main()

trial = 1


AerError: 'unknown instruction: c 8^1 mod 15'

In [84]:
# QPE 회로 그리기
a = 7
phase, qc = qpe_amod15(a)
qc.draw()

AerError: 'unknown instruction: c 7^1 mod 15'

In [60]:
# Modular Power 회로 그리기
a = 2
power = 4
U = QuantumCircuit(4)
for iteration in range(power):
    if a in [2, 13]:
        U.swap(0, 1)
        U.swap(1, 2)
        U.swap(2, 3)
    if a in [7, 8]:
        U.swap(2, 3)
        U.swap(1, 2)
        U.swap(0, 1)
    if a == 11:
        U.swap(1, 3)
        U.swap(0, 2)
    if a in [7, 11, 13]:
        for q in range(4):
            U.x(q)
U.draw()

q_0: ─X─────X─────X─────X───────
      │     │     │     │       
q_1: ─X──X──X──X──X──X──X──X────
         │     │     │     │    
q_2: ────X──X──X──X──X──X──X──X─
            │     │     │     │ 
q_3: ───────X─────X─────X─────X─

In [61]:
# I-QFT 회로 그리기

qc = qft_dagger(3)
qc.draw()

┌───┐                                        
q_0: ─X─┤ H ├─■──────────────■───────────────────────
      │ └───┘ │P(-π/2) ┌───┐ │                       
q_1: ─┼───────■────────┤ H ├─┼─────────■─────────────
      │                └───┘ │P(-π/4)  │P(-π/2) ┌───┐
q_2: ─X──────────────────────■─────────■────────┤ H ├
                                                └───┘